In [1]:
from selenium import webdriver
import pandas as pd

##### Gmarket 베스트 셀러 데이터 스크래핑 
- 전체 200개중 상위 10개 출력
- pandas 데이터를 분석할수 있는 python 라이브러리
    - pip3 install pandas
    - 데이터 프레임 생성 : df = pd.DataFrame(columns=["A","B"])
    - 데이터 추가 : df.loc[len(df)] = [{"A":1,"B":2}, {"A":3,"B":4}]

In [2]:
def get_gmarket_bestsellers(number=10):
    
    def to_int(price):
        return int(price.replace(",","").replace("원",""))
    
    driver =  webdriver.Chrome()
    driver.get('http://corners.gmarket.co.kr/Bestsellers')

    bestlist_selector = driver.find_elements_by_css_selector("#gBestWrap .best-list")[1]
    items = bestlist_selector.find_elements_by_css_selector("li")[:number]
    
    df = pd.DataFrame(columns=["상품명","원가","판매가","이미지","상품링크"])
    
    for item in items:

        idx = item.find_element_by_css_selector("p").text # 순번
        img_link = item.find_element_by_css_selector("img").get_attribute("src") # 이미지 URL
        item_link = item.find_element_by_css_selector(".itemname").get_attribute("href") # 아이템 URL
        content = item.find_element_by_css_selector(".itemname").text # 아이템 텍스트
        o_price = item.find_element_by_css_selector(".item_price .o-price").text # 오리지날 가격
        s_price = item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[0] # 판매 가격
        o_price = o_price if o_price else s_price # 원가 데이터가 없는경우 판매가를 대입
        
        data = {"상품명": content,
                "원가": to_int(o_price), 
                "판매가": to_int(s_price),
                "이미지": img_link, 
                "상품링크": item_link, }
        
        # Pandas 데이터 입력
        df.loc[len(df)] = data
        
    driver.close()
    return df

In [3]:
%%time
result = get_gmarket_bestsellers(200)
result.tail()

CPU times: user 1.62 s, sys: 84.6 ms, total: 1.71 s
Wall time: 30.1 s


In [4]:
# Pandas로 할인율 계산
result["할인율"] = 100 - (result["판매가"] / result["원가"]) * 100
result["할인율"] = result["할인율"].astype(dtype="float64")
result["할인율"] = round(result["할인율"], 2)
result = result.sort_values("할인율", ascending=False).reset_index().rename(columns={"index": "rank"})
result.head()

,rank,상품명,원가,판매가,이미지,상품링크,할인율
0,196,[쁘리엘르]100% 순면차렵이불/패드세트/마이크로화이바/피치스킨,99900,19900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,80.08
1,63,크록스키즈 역시즌특가 50종택1 (G180217),53000,15900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
2,158,엄청싸다 신상 gold 반지 커플링 도매가판매,53000,15900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
3,187,국산 유아.아동.주니어 내의/내복/실내복/잠옷/신상,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
4,75,삠뽀요/봄신상 원피스9900원 베이비 우주복 돌복,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00


In [22]:
from selenium import webdriver
import pickle
import time

In [24]:
main_url = "http://naver.com"
mail_url = "http://mail.naver.com"

In [25]:
# 브라우져 오픈 및 메인 사이트 이동
driver = webdriver.Chrome()
driver.get(main_url)

In [26]:
#패스워드 저장하기
pw = ""
file = open("pw.pickle", "wb")
pickle.dump(pw, file)

In [29]:
# 아이디 패스워드 입력
pw = pickle.load(open("pw.pickle", "rb")) # pw 변수에 본인의 패스워드 입력
driver.find_element_by_css_selector( "#id" ).send_keys( "hosuano" )
driver.find_element_by_css_selector( "#pw" ).send_keys( pw )

In [30]:
# 로그인 버튼 클릭
driver.find_element_by_css_selector("span.btn_login input[type=submit]").click()

In [31]:
# 메일 페이지로 이동
driver.get(mail_url)

In [35]:
# 메일 갯수
total_mail = driver.find_element_by_css_selector("#headTotalNum").text
total_mail = int(total_mail)
total_mail

98

In [36]:
# 안읽은 메일 갯수
not_read_mail = driver.find_element_by_css_selector("#headUnreadNum").text
not_read_mail = int(not_read_mail)
not_read_mail

97

In [37]:
# 메일 제목, 보낸사람 이름 가져오기
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  15


,제목,보낸사람,메일링크
10,"Joon Seong Lee님, 클릭 한 번으로 Facebook에 다시 로그인하세요",Facebook,https://mail.naver.com/read/popup/?nMailId=11772
11,yessign 공인인증서 개인정보 이용내역 안내,금융결제원,https://mail.naver.com/read/popup/?nMailId=11771
12,Why is Japan so jealous of South Korea?,Quora Digest,https://mail.naver.com/read/popup/?nMailId=11770
13,"블렌딩티 힛더티, 비트파인더의 어웨어, 블루팬넷, 크리에이스랩, 버트스튜디오 소식",데일리 로켓펀치,https://mail.naver.com/read/popup/?nMailId=11769
14,(스타일맨) 개인정보 처리방침 변경 사전 안내,스타일맨,https://mail.naver.com/read/popup/?nMailId=11768


In [38]:
# marked 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li.mark")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  0


,제목,보낸사람,메일링크


In [39]:
# 읽은 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li:not(.notRead)")
print("total mails: ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

total mails:  0


,제목,보낸사람,메일링크


In [40]:
# 로그아웃
logout_url = driver.find_element_by_css_selector(".gnb_btn_login").get_attribute("href")
driver.get(logout_url)
# 브라우져 닫기
driver.close()